In [ ]:
import os
import re
import json
import ast
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
import numpy as np
import ast
import pandas as pd


In [ ]:
path = '../self_contra/wino_bias_reason_answer_for_generated_question_by_mistral_full/'
files = os.listdir(path)
files


In [ ]:
if '.DS_Store' in files:
    files.remove('.DS_Store')
len(files)

In [ ]:
# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')


In [ ]:
files[0]

In [ ]:
data = pd.read_csv(path + files[0])
data.columns

In [ ]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def get_emb(question_dict):
    encoded_input = tokenizer(question_dict, padding=True, truncation=False, return_tensors='pt')
    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

    return sentence_embeddings


In [ ]:
for j, file_name in enumerate(files):
    print(j, file_name)
    data = pd.read_csv(path + file_name)
    sim_genrea_reason = []
    sim_predans_gt = []
    sim_predans_pred = []
    
    for i, row in data.iterrows():
        gen_reason = row['reason_for_generated']
        reason = row['reasoning']
        gen_pred = row['pred_for_generated']
        gt = row['ground_truth']
        pred = row['prediction']
        
        temp = [gen_reason, reason, gen_pred, gt, pred]
        sentence_embeddings = get_emb(temp)
        
        similarity_reasons = cosine_similarity(sentence_embeddings[0].cpu().numpy().reshape(1, -1), sentence_embeddings[1].cpu().numpy().reshape(1, -1))[0][0]
        similarity_gt = cosine_similarity(sentence_embeddings[2].cpu().numpy().reshape(1, -1), sentence_embeddings[3].cpu().numpy().reshape(1, -1))[0][0]
        similarity_pred = cosine_similarity(sentence_embeddings[2].cpu().numpy().reshape(1, -1), sentence_embeddings[4].cpu().numpy().reshape(1, -1))[0][0]
        
        sim_genrea_reason.append(similarity_reasons)
        sim_predans_gt.append(similarity_gt)
        sim_predans_pred.append(similarity_pred)
        
    data['similarity_bt_reasons'] = sim_genrea_reason
    data['similarity_bt_predans_gt'] = sim_predans_gt
    data['similarity_bt_predans_pred'] = sim_predans_pred
    
    data.to_csv('../self_contra/wino_bias_reason_similarity_for_generated_by_mistral/' + file_name)
        
            